# Ejercicio: Ingeniería de X entradas (características) y Regresión Polinómica


## Objetivos
En este ejercicio usted:
- explorará la ingeniería de X entradas (características) y la regresión polinómica, lo que le permite usar la maquinaria de la regresión lineal para ajustar funciones muy complicadas, incluso muy no lineales.


## Herramientas
Utilizará la función desarrollada en ejercicios anteriores así como matplotlib y NumPy. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ejerc_utils_multi import zscore_normalize_features, run_gradient_descent_feng
np.set_printoptions(precision=2)  # precisión de visualización reducida en arreglos numpy

<a name='FeatureEng'></a>
# Visión General de Ingeniería de X entradas (características) y Regresión Polinómica

De forma predeterminada, la regresión lineal proporciona un medio para construir modelos de la forma:
$$f_{\mathbf{w},b} = w_0x_0 + w_1x_1+ ... + w_{n-1}x_{n-1} + b \tag{1}$$ 
¿Qué pasa si sus X entradas (características)/datos son no lineales o son combinaciones de X entradas (características)? Por ejemplo, los precios de viviendas no tienden a ser lineales con el área habitable (metros) sino que penalizan casas muy pequeñas o muy grandes, resultando en las curvas mostradas en la imagen de arriba. ¿Cómo podemos usar la maquinaria de la regresión lineal para ajustar esta curva? Recuerde, la 'maquinaria' que tenemos es la capacidad de modificar los parámetros $\mathbf{w}$, $\mathbf{b}$ en (1) para 'ajustar' la ecuación a los datos de entrenamiento. Sin embargo, ningún ajuste de $\mathbf{w}$,$\mathbf{b}$ en (1) logrará un ajuste a una curva no lineal.


<a name='PolynomialFeatures'></a>
## X entradas (características) polinómicas

Arriba estábamos considerando un escenario donde los datos eran no lineales. Probemos usando lo que sabemos hasta ahora para ajustar una curva no lineal. Comenzaremos con una cuadrática simple: $y = 1+x^2$

Usted ya está familiarizado con todas las rutinas que estamos usando. Están disponibles en el archivo lab_utils.py para revisión. Usaremos [`np.c_[..]`](https://numpy.org/doc/stable/reference/generated/numpy.c_.html) que es una rutina de NumPy para concatenar a lo largo del límite de columna.

In [ ]:
# crear datos objetivo
x = np.arange(0, 20, 1)
y = 1 + x**2
X = x.reshape(-1, 1)

model_w,model_b = run_gradient_descent_feng(X,y,iterations=1000, alpha = 1e-2)

plt.scatter(x, y, marker='x', c='r', label="Valor real"); plt.title("sin ingeniería de X entradas (características)")
plt.plot(x,X@model_w + model_b, label="Valor predicho");  plt.xlabel("X"); plt.ylabel("y"); plt.legend(); plt.show()

Bien, como era de esperar, no es un buen ajuste. Lo que se necesita es algo como $y= w_0x_0^2 + b$, o una **X entrada (característica) polinómica**.
Para lograr esto, puede modificar los *datos de entrada* para *ingenierizar* las X entradas (características) necesarias. Si reemplaza los datos originales por una versión que eleva al cuadrado el valor de $x$, entonces puede lograr $y= w_0x_0^2 + b$. Probémoslo. Reemplace `X` por `X**2` abajo:

In [ ]:
# crear datos objetivo
x = np.arange(0, 20, 1)
y = 1 + x**2

# Ingeniería de X entradas (características) 
X = x**2      #<-- X entrada (característica) ingenierizada agregada

In [ ]:
X = X.reshape(-1, 1)  #X debe ser una matriz 2-D
model_w,model_b = run_gradient_descent_feng(X, y, iterations=10000, alpha = 1e-5)

plt.scatter(x, y, marker='x', c='r', label="Valor real"); plt.title("X entrada (característica) x**2 agregada")
plt.plot(x, np.dot(X,model_w) + model_b, label="Valor predicho"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

¡Genial! ajuste casi perfecto. Observe los valores de $\mathbf{w}$ y b impresos justo arriba del gráfico: `w,b encontrados por descenso de gradiente: w: [1.], b: 0.0490`. El descenso de gradiente modificó nuestros valores iniciales de $\mathbf{w},b $ a (1.0,0.049) o un modelo de $y=1*x_0^2+0.049$, muy cerca de nuestro objetivo de $y=1*x_0^2+1$. Si lo ejecuta por más tiempo, podría ser una mejor coincidencia. 

### Selección de X entradas (características)
<a name='GDF'></a>
Arriba, sabíamos que se requería un término $x^2$. No siempre es obvio qué X entradas (características) se requieren. Se podrían agregar varias X entradas (características) potenciales para intentar encontrar las más útiles. Por ejemplo, ¿qué pasaría si en su lugar intentáramos: $y=w_0x_0 + w_1x_1^2 + w_2x_2^3+b$ ? 

Ejecute las siguientes celdas. 

In [ ]:
# crear datos objetivo
x = np.arange(0, 20, 1)
y = x**2

# ingeniería de X entradas (características) .
X = np.c_[x, x**2, x**3]   #<-- X entrada (característica) ingenierizada agregada

In [ ]:
model_w,model_b = run_gradient_descent_feng(X, y, iterations=10000, alpha=1e-7)

plt.scatter(x, y, marker='x', c='r', label="Valor real"); plt.title("X entradas (características) x, x**2, x**3")
plt.plot(x, X@model_w + model_b, label="Valor predicho"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

Observe el valor de $\mathbf{w}$, `[0.08 0.54 0.03]` y b es `0.0106`. Esto implica que el modelo después de ajustar/entrenar es:
$$ 0.08x + 0.54x^2 + 0.03x^3 + 0.0106 $$
El descenso de gradiente ha enfatizado los datos que mejor se ajustan a los datos de $x^2$ aumentando el término $w_1$ en relación con los demás. Si lo ejecutara durante mucho tiempo, continuaría reduciendo el impacto de los otros términos. 

>El descenso de gradiente está eligiendo las X entradas (características) 'correctas' para nosotros al enfatizar su parámetro asociado

Revisemos esta idea:
- un valor de peso menor implica una X entrada (característica) menos importante/correcta, y en el extremo, cuando el peso se vuelve cero o muy cercano a cero, la X entrada (característica) asociada no es útil para ajustar el modelo a los datos.
- arriba, después de ajustar, el peso asociado a la X entrada (característica) $x^2$ es mucho mayor que los pesos para $x$ o $x^3$ ya que es la más útil para ajustar los datos. 

### Una Vista Alternativa
Arriba, las X entradas (características) polinómicas se eligieron en función de qué tan bien coincidían con los datos objetivo. Otra forma de pensar en esto es notar que seguimos usando regresión lineal una vez que hemos creado nuevas X entradas (características). Dado eso, las mejores X entradas (características) serán lineales respecto al objetivo. Esto se entiende mejor con un ejemplo. 

In [ ]:
# crear datos objetivo
x = np.arange(0, 20, 1)
y = x**2

# ingeniería de X entradas (características) .
X = np.c_[x, x**2, x**3]   #<-- X entrada (característica) ingenierizada agregada
X_features = ['x','x^2','x^3']

In [ ]:
fig,ax=plt.subplots(1, 3, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X[:,i],y)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("y")
plt.show()

Arriba, es claro que la X entrada (característica) $x^2$ mapeada contra el valor objetivo $y$ es lineal. La regresión lineal puede entonces generar fácilmente un modelo usando esa X entrada (característica).

### Escalado de X entradas (características)
Como se describió en el ejercicio anterior, si el conjunto de datos tiene X entradas (características) con escalas significativamente diferentes, se debe aplicar escalado de X entradas (características) para acelerar el descenso de gradiente. En el ejemplo anterior, hay $x$, $x^2$ y $x^3$ que naturalmente tendrán escalas muy diferentes. Apliquemos la normalización z-score a nuestro ejemplo.

In [ ]:
# crear datos objetivo
x = np.arange(0,20,1)
X = np.c_[x, x**2, x**3]
print(f"Rango pico a pico por columna en X original: {np.ptp(X,axis=0)}")

# agregar normalización por la media 
X = zscore_normalize_features(X)     
print(f"Rango pico a pico por columna en X normalizado: {np.ptp(X,axis=0)}")

Ahora podemos intentar de nuevo con un valor de alpha más agresivo:

In [ ]:
x = np.arange(0,20,1)
y = x**2

X = np.c_[x, x**2, x**3]
X = zscore_normalize_features(X) # features = características (X entradas)

model_w, model_b = run_gradient_descent_feng(X, y, iterations=100000, alpha=1e-1)

plt.scatter(x, y, marker='x', c='r', label="Valor real"); plt.title("X entradas (características) normalizadas x x**2, x**3")
plt.plot(x,X@model_w + model_b, label="Valor predicho"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

El escalado de X entradas (características) permite que esto converja mucho más rápido.   
Observe nuevamente los valores de $\mathbf{w}$. El término $w_1$, que es el término $x^2$, es el más enfatizado. El descenso de gradiente prácticamente ha eliminado el término $x^3$.

### Funciones Complejas
Con la ingeniería de X entradas (características), incluso funciones bastante complejas pueden ser modeladas:

In [ ]:
x = np.arange(0,20,1)
y = np.cos(x/2)

X = np.c_[x, x**2, x**3,x**4, x**5, x**6, x**7, x**8, x**9, x**10, x**11, x**12, x**13]
X = zscore_normalize_features(X) 

model_w,model_b = run_gradient_descent_feng(X, y, iterations=1000000, alpha = 1e-1)

plt.scatter(x, y, marker='x', c='r', label="Valor real"); plt.title("X entradas (características) normalizadas x x**2, x**3")
plt.plot(x,X@model_w + model_b, label="Valor predicho"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()



## ¡Felicitaciones!
En este ejercicio usted:
- aprendió cómo la regresión lineal puede modelar funciones complejas, incluso altamente no lineales, usando ingeniería de X entradas (características)
- reconoció que es importante aplicar escalado de características al hacer ingeniería de X entradas (características)